In [ ]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

from glob import glob
import re

import matplotlib.pyplot as plt
from utils import compute_w_ratio, draw, get_points
from os.path import basename
import json

In [ ]:
def parse(s):
    for w in ['high', 'medium', 'low']:
        if s.split('/')[-1] == f'avgimg_wr_{w}.png':
            return w, 0
    w, m = re.findall(r".*(low|medium|high)_\((.\d+)\).png$", s)[0]
    return w, int(m)

In [ ]:
IMAGE_FILES = glob('../data/human-in-the-loop/stimuli/*/*.png')
meshs = {}
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
      for idx, file in enumerate(IMAGE_FILES):
            print(file, end='.')
            image = cv2.imread(file)
            # Convert the BGR image to RGB before processing.
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            t = parse(file)
            meshs[t] = results

            # Print and draw face mesh landmarks on the image.
            if not results.multi_face_landmarks:
                  continue

In [ ]:
with open('../data/processed/stimulus_wr.json', 'w') as fh:
    json.dump({basename(fn): compute_w_ratio(meshs[parse(fn)]) for fn in IMAGE_FILES}, fh)

In [ ]:
for fn in IMAGE_FILES:
    t = parse(fn)
    results = meshs[t]
    image = cv2.imread(fn)
    image = draw(image, results)

    w_ratio = compute_w_ratio(results)
    p_lefteye, p_righteye, p_left, p_right = get_points(results)

    height, width, channels = image.shape
    plt.title(f"{t[0]} ({t[1]}), gemessen: {w_ratio:.4f}")
    plt.imshow(image[:, :, ::-1])
    plt.plot(p_lefteye[0]*width, p_lefteye[1]*height, 'x')
    plt.plot(p_righteye[0]*width, p_righteye[1]*height, 'x')
    plt.plot(p_left[0]*width, p_left[1]*height, 'x')
    plt.plot(p_right[0]*width, p_right[1]*height, 'x')
    plt.show()

In [ ]:
import pandas as pd
df = pd.DataFrame(index=[-100, -66, -33, 0, 33, 66, 100], columns=['low', 'medium', 'high'])

for fn in IMAGE_FILES:
    t = parse(fn)
    results = meshs[t]

    w_ratio = compute_w_ratio(results)
    df.loc[t[1], t[0]] = w_ratio
#     print(t, w_ratio)

In [ ]:
df

In [ ]:
# plt.vlines(df.low, -1, 1, colors='r', label='low')
# plt.vlines(df.medium, -1, 1, colors='g', label='medium')
# plt.vlines(df.high, -1, 1, colors='b', label='high')
plt.plot(df.low, 7*[0], 'o', label='low', alpha=0.5)
plt.plot(df.medium, 7*[0], 'o', label='medium', alpha=0.5)
plt.plot(df.high, 7*[0], 'o', label='high', alpha=0.5)
plt.legend()